In [20]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib import dates as d
import datetime as dt
import time

from functools import reduce

In [6]:
np.exp(0)

1.0

In [124]:
def Specific_humidity(T, p, RH):
    # unit of specific humidity: kg/kg
    # pressure in hPa = mbar
    # Temperature in Celsius
    # RH = relative humidity (%)
    
    # q denotes specific humidity
    # formula: q = qs * RH = 0.622 * es/p * RH
    # with es the saturated vapor pressure, formula: Buck equation, Wikipedia
    q = RH/100 * 0.622 * 6.1121 * 1./p *np.exp((18.678 - T/234.5)*(T/(257.14 + T)))
    return(q)

In [125]:
Specific_humidity(4, 600.1, 30)

0.0025293934354107686

In [14]:
# prepare CFHT observational data for merging
def mes_prep(CFHT_hourly, parameter = None):
    CFHT_hourly = CFHT_hourly.rename(columns={'Unnamed: 0': 'time'})
    # change the format of the times column to datetime format
    CFHT_hourly['time'] = pd.to_datetime(CFHT_hourly['time']) 

    #check the format
    print(CFHT_hourly['time'].dtype)
    #print(CFHT_hourly['time'][0])

    #from HST to UTC (+10 hours)
    CFHT_hourly['time'] = CFHT_hourly['time'] + dt.timedelta(hours=10)
    
    #set index 
    CFHT_hourly.set_index('time', inplace=True)
    
    if parameter == 'temperature(C)':
        #filter out values exactly equal to 0
        mask_T = (CFHT_hourly['temperature(C)'] != 0)
        CFHT_hourly = CFHT_hourly[mask_T]
        print('masked')

    # create a new column consisting of the cycle parameter of the correspondend entry
    #for seasonal cycle (12 months), create column with "months"
    #CFHT_hourly['months'] = pd.DatetimeIndex(CFHT_hourly.index).month                                            

    #for diurnal cycle (24 hours), create column with "hours"
    #CFHT_hourly['hours'] = pd.DatetimeIndex(CFHT_hourly.index).hour
    
    return(CFHT_hourly)

In [9]:
#observational data from MaunaKea (CFHT), temperature, RH, pressure
CFHT_T_hourly = pd.read_csv('/home/caroline/Dropbox/Astroclimate Project/Mauna_Kea/CFHT/Temperature/downsampled_CFHT_T_1991to2018_hourly_means.csv')
CFHT_RH_hourly = pd.read_csv('/home/caroline/Dropbox/Astroclimate Project/Mauna_Kea/CFHT/pressure_levels_600to750hPa/RH/R/downsampled_masked_RH_1991to2018_hourly_means.csv')
CFHT_P_hourly = pd.read_csv('/home/caroline/Dropbox/Astroclimate Project/Mauna_Kea/CFHT/pressure/downsampled_Pressure_hourly.csv')

In [18]:
CFHT_P_hourly = mes_prep(CFHT_P_hourly)
CFHT_RH_hourly = mes_prep(CFHT_RH_hourly)
CFHT_T_hourly = mes_prep(CFHT_T_hourly, 'temperature(C)')

datetime64[ns]
datetime64[ns]
masked


In [21]:
# merge datasets, filter nans
    
df_list = [CFHT_T_hourly, CFHT_P_hourly, CFHT_RH_hourly, ]
df_merged = reduce(lambda left, right: pd.merge(left, right, left_on='time', right_on='time', how='outer'), df_list)

# delete rows containing NaN
df_merged_nonan = df_merged.dropna(axis='rows', how='any', thresh=None, subset=None, inplace=False)

In [24]:
df_merged_nonan

,temperature(C),pressure (mb),relative_humidity(%)
time,,,
2000-01-01 10:00:00,-0.736667,612.560000,27.386667
2000-01-01 11:00:00,-0.481667,612.330000,25.248333
2000-01-01 12:00:00,-0.451667,611.941667,25.190000
2000-01-01 13:00:00,-0.358333,611.471667,25.373333
2000-01-01 14:00:00,-0.110000,611.638333,23.635000
...,...,...,...
2020-01-17 08:00:00,-0.508500,614.150000,98.466667
2020-01-17 09:00:00,-0.339333,614.043333,96.383333
2020-01-17 10:00:00,-0.553333,613.805000,98.433333


In [92]:
df_merged_nonan = df_merged_nonan.rename(columns={'T': 'Temp'})
#df_merged_nonan = df_merged_nonan.rename(columns={'temperature(C)': 'T'})

In [42]:
df_merged_nonan = df_merged_nonan.rename(columns={'relative_humidity(%)': 'RH'})

In [41]:
df_merged_nonan = df_merged_nonan.rename(columns={'pressure (mb)': 'P'})

In [43]:
df_merged_nonan

,T,P,RH
time,,,
2000-01-01 10:00:00,-0.736667,612.560000,27.386667
2000-01-01 11:00:00,-0.481667,612.330000,25.248333
2000-01-01 12:00:00,-0.451667,611.941667,25.190000
2000-01-01 13:00:00,-0.358333,611.471667,25.373333
2000-01-01 14:00:00,-0.110000,611.638333,23.635000
...,...,...,...
2020-01-17 08:00:00,-0.508500,614.150000,98.466667
2020-01-17 09:00:00,-0.339333,614.043333,96.383333
2020-01-17 10:00:00,-0.553333,613.805000,98.433333


In [112]:
#calculate specific humidity, write to csv

# initialize new column (veeery fast!)
df_merged_nonan['specific_humidity'] = np.vectorize(Specific_humidity)(df_merged_nonan.Temp, df_merged_nonan.P, df_merged_nonan.RH)
#df_merged_nonan['specific_humidity'] = df_merged_nonan.apply(Specific_humidity(df_merged_nonan.T, df_merged_nonan.P, df_merged_nonan.RH), axis = 1)

In [113]:
df_merged_nonan

,Temp,P,RH,specific_humidity
time,,,,
2000-01-01 10:00:00,-0.736667,612.560000,27.386667,0.001611
2000-01-01 11:00:00,-0.481667,612.330000,25.248333,0.001514
2000-01-01 12:00:00,-0.451667,611.941667,25.190000,0.001514
2000-01-01 13:00:00,-0.358333,611.471667,25.373333,0.001537
2000-01-01 14:00:00,-0.110000,611.638333,23.635000,0.001457
...,...,...,...,...
2020-01-17 08:00:00,-0.508500,614.150000,98.466667,0.005874
2020-01-17 09:00:00,-0.339333,614.043333,96.383333,0.005822
2020-01-17 10:00:00,-0.553333,613.805000,98.433333,0.005856


In [122]:
# there are very high values sometimes, investigate!

mask = (df_merged_nonan['specific_humidity'] < 0.1)
df_merged_nonan_masked = df_merged_nonan[mask]

In [127]:
mask_extreme = (df_merged_nonan['specific_humidity'] > 0.1)
df_merged_nonan[mask_extreme]

,Temp,P,RH,specific_humidity
time,,,,
2009-07-27 22:00:00,-825.003333,619.108333,18.727273,1.162471e+11
2009-09-22 01:00:00,-1659.008333,616.508333,34.800000,3.692732e+10


In [123]:
df_merged_nonan_masked.to_csv('Specific_humidity_CFHT_masked_2000to2019.csv')

In [109]:
Specific_humidity(-0.481667, 612.330000, 25.248333)

0.0015135721390234379

In [110]:
df_test['specific_humidity_3'] = np.vectorize(Specific_humidity)(df_test.Temp, df_test.P, df_test.RH)

/home/caroline/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [111]:
df_test

,Temp,P,RH,specific_humidity,specific_humidity_2,specific_humidity_3
time,,,,,,
2000-01-01 10:00:00,-0.736667,612.560000,27.386667,0.001611,0.001611,0.001611
2000-01-01 11:00:00,-0.481667,612.330000,25.248333,0.001611,0.001514,0.001514
2000-01-01 12:00:00,-0.451667,611.941667,25.190000,0.001611,0.001514,0.001514
2000-01-01 13:00:00,-0.358333,611.471667,25.373333,0.001611,0.001537,0.001537
2000-01-01 14:00:00,-0.110000,611.638333,23.635000,0.001611,0.001457,0.001457
